In [16]:
from DataFetcher import DataFetcher
from USTs import USTs
from Curves import ParCurves
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go

In [2]:
auctions = DataFetcher().fetch_auction_data()
prices = DataFetcher().fetch_historical_UST_data(date=datetime.datetime(year=2025, month=6, day=26))

In [3]:
ust_set = USTs(auction_data=auctions,price_data=prices).get_current_UST_set(settlement_date=datetime.datetime(2025, 6, 27),
                                                                            get_ytms=True,
                                                                            include_FRNs=False,
                                                                            include_TIPS=False)
ust_set

Merged auction and price data successfully
No missing or excess data
All CUSIPs are identical between DataFrames


,Cusip,Security type,Rate,Maturity date,End of day,Original security term,Issue date,Currently outstanding,Time to expiry,EOD YTM,Rank
2,912828XZ8,Note,2.750,2025-06-30,99.968750,7-Year,2018-07-02,NaN,0.010959,6.559941,84
1,91282CHL8,Note,4.625,2025-06-30,99.968750,2-Year,2023-06-30,NaN,0.010959,8.400867,24
0,912828ZW3,Note,0.250,2025-06-30,99.968750,5-Year,2020-06-30,NaN,0.010959,4.078317,58
3,912797PU5,Bill,0.000,2025-07-01,99.954778,17-Week,2025-03-04,1.354470e+11,0.013699,4.128374,17
4,912797NX1,Bill,0.000,2025-07-03,99.932333,26-Week,2025-01-02,1.571050e+11,0.019178,4.119197,19
...,...,...,...,...,...,...,...,...,...,...,...
451,912810UA4,Bond,4.625,2054-05-15,96.687500,30-Year,2024-05-15,5.085000e+10,28.904110,4.840632,5
452,912810UC0,Bond,4.250,2054-08-15,90.906250,30-Year,2024-08-15,5.196500e+10,29.156164,4.833356,4
453,912810UE6,Bond,4.500,2054-11-15,94.875000,30-Year,2024-11-15,5.322900e+10,29.408219,4.829236,3
455,912810UG1,Bond,4.625,2055-02-15,96.906250,30-Year,2025-02-18,5.423000e+10,29.660274,4.821016,2


In [39]:
filtered_set = ust_set[(ust_set['Time to expiry'] > 90/365) & (ust_set['Security type'] != 'Bill')]
spline = ParCurves(filtered_set).cubicspline_interpolator()
curve_x, curve_y = np.arange(0.25, 30, 0.1), spline(np.arange(0.25, 30, 0.1))

years = ['2', '3', '5', '7', '10', '20', '30']
terms = [tenor + '-Year' for tenor in years] # Ordered list

customdata = filtered_set.to_numpy()
fig = go.Figure()
hovertemplate = (
    "<b>CUSIP:</b> %{customdata[0]}<br>"+
    "<b>YTM:</b> %{y:.3%}<br>"+
    "<b>Price:<b> %{customdata[4]:.4f}<br>"+
    "<b>Maturity date:</b> %{customdata[3]|%Y-%m-%d}<br>"+
    "<b>Issue date:</b> %{customdata[6]|%Y-%m-%d}<br>"
)
for tenor in terms:
    subset = filtered_set[filtered_set['Original security term'] == tenor]
    fig.add_trace(go.Scatter(
        x=subset['Time to expiry'],
        y=subset['EOD YTM']/100,
        mode='markers',
        name=tenor,
        hovertemplate=hovertemplate,
        customdata=customdata,
        text=subset['EOD YTM'],
        marker=dict(
            size=6
        )
    ))
fig.add_trace(go.Scatter(
    x=curve_x,
    y=curve_y/100,
    mode='lines',
    name='Cubic spline'
))
fig.update_layout(
    title='US Treasury Yield Curve',
    width=1100,
    height=600,
    margin=dict(l=20, r=20, t=50, b=20),
    xaxis_title='Years to maturity',
    yaxis_title='End-of-Day Yield to Maturity (YTM)',
    yaxis_tickformat='.2%',
    legend_title_text='Security Tenors',
    template='plotly_dark'
)
fig.show()
